In [1]:
import sys
sys.path.append('../../lib/')

This script uses tweet jsons. It takes all of the tweet ids and gets the image url and user description. 

In [2]:
from analysis import *
import json
import requests
import time
import shutil

In [3]:
with open('../../key.json') as f:
	key = json.load(f)
	bearer_token = key['BEARER_TOKEN']

In [4]:
tweets_list_possable_bad = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/posable_bad_tweets.json"))
tweets_list_good = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/300_random_users_first_tweet.json"))
tweets_list_false_positives = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/false_postives_from_v3_ivory.json"))
seed_user_good_tweets = analysis.create_tweet_list_from_loaded_json(analysis.universal_data_loader("../../data/dataset/100_random_tweets_from_seed_users.json"))

tweets_list = tweets_list_possable_bad + tweets_list_good + tweets_list_false_positives + seed_user_good_tweets

In [5]:
print("Number of bad tweets: " + str(len(tweets_list_possable_bad)))
print("Number of good tweets: " + str(len(tweets_list_good)))
print("Number of false positives: " + str(len(tweets_list_false_positives)))
print("Number of seed user tweets: " + str(len(seed_user_good_tweets)))
print("Number of tweets: " + str(len(tweets_list)))

Number of bad tweets: 168
Number of good tweets: 240
Number of false positives: 131
Number of seed user tweets: 100
Number of tweets: 639


In [6]:
# Check to make sure all the tweets are unique
tweet_ids = {}
for tweet in tweets_list:
	if analysis.get_tweet_id(tweet) in tweet_ids:
		print(analysis.get_tweet_id(tweet))
		tweet_ids[analysis.get_tweet_id(tweet)] += 1
	else:
		tweet_ids[analysis.get_tweet_id(tweet)] = 1


In [7]:
print(len(tweets_list))

639


In [8]:
# Get timeline
def get_tweet(tweet_id):
	# Create the url
	url = 'https://api.twitter.com/2/tweets/{}'.format(tweet_id) + '?expansions=attachments.media_keys,referenced_tweets.id.author_id&&media.fields=url&&tweet.fields=id&&user.fields=description'
	# Create request headers
	headers = {
		'authorization': 'Bearer ' + bearer_token,
		'user-agent': 'v2FilteredStreamPython',
	}
	# Make the request
	response = requests.get(url, headers=headers)
	return response.json()

In [9]:
tweets_with_images = {}
tweets_with_images["tweets"] = []
# Go through each tweet
for tweet in tweets_list:
    try:
        tweet_response = get_tweet(analysis.get_tweet_id(tweet))
        if tweet_response['data']:
            tweets_with_images["tweets"].append(tweet_response)
    except:
        print(analysis.get_tweet_text(tweet)) 
    time.sleep(10)

@SF_Moro Não, desista
@damnarbor I'll allow it.  But you also have to look at the hideous 70s (?) era apartments in the OWS to see how Blah this can be, and how bad the response can be (OWS Association is absurd). Is there a good inbetween?
@moxie Relevant you web3 point that centralization moves faster than protocols https://t.co/AoWazygKE5


In [10]:
# Write to file
with open('../../data/dataset/tweets_with_images.json', 'w') as f:
	json.dump(tweets_with_images, f)

In [11]:
json_image_path = '../../data/dataset/tweets_with_images.json'
# load json file
with open(json_image_path) as json_file:
	data = json.load(json_file)

In [12]:
print(len(data["tweets"]))

636


In [13]:
for tweet in data["tweets"]:
    image_count = 0
    try:
        for image in tweet['includes']['media']:
            image_url = image['url']
            print(image_url)
            # Download image
            r = requests.get(image_url, stream=True)
            
            # Get image type from url
            image_type = image_url.split('.')[-1]
            if r.status_code == 200:
                with open('../../data/dataset/images/' + str(tweet['data']['id']) + "_" + str(image_count) + "." + image_type, 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f)
            image_count += 1
    except:
        print("Error id: " + str(tweet['data']['id']))


https://pbs.twimg.com/media/EOfhQPyXkAI7um2.png
https://pbs.twimg.com/media/EOUbGtNWoAMuDNi.jpg
https://pbs.twimg.com/media/EN1R8i7XsAIAEED.png
https://pbs.twimg.com/media/ENXuDhRXkAUS2QJ.png
https://pbs.twimg.com/media/ENIudOBWkAAmGOE.jpg
https://pbs.twimg.com/media/EKcxagfWwAAahd4.png
https://pbs.twimg.com/media/EKSGJDXW4AIp7KZ.png
https://pbs.twimg.com/media/EKDinLDWwAA9wVB.png
https://pbs.twimg.com/media/EJ-Eo9oW4AY0Udt.png
https://pbs.twimg.com/media/EJzq48vWwAAgMhv.png
https://pbs.twimg.com/media/EJuorSOXsAAUvJf.png
https://pbs.twimg.com/media/EJpeuidWoAEqX_C.png
https://pbs.twimg.com/media/EJkriQJXkAEhqSG.png
https://pbs.twimg.com/media/EJgT5OTXsAEcwga.jpg
https://pbs.twimg.com/media/EJUvfbBXsAAGNZe.png
https://pbs.twimg.com/media/EJPndq-XUAUIuYZ.png
https://pbs.twimg.com/media/EJFPWrNXsAAVOZ0.png
https://pbs.twimg.com/media/EJA72oxXUAEFeHY.png
https://pbs.twimg.com/media/EI7XlPIW4AAzxyk.png
https://pbs.twimg.com/media/EI1sNSVWoAAB1IJ.png
https://pbs.twimg.com/media/EIr2qiOXYAA3